In [1]:
import numpy as np
import pydicom
import json
import os
import shutil
import sys
import random
from matplotlib import image
from scipy.ndimage import label
from zipfile import ZipFile
import re
import pandas as pd
from skimage.transform import resize

In [ ]:
# this varibale should be set as where your train.zip, validate.zip, test.zip store
data_path = '.'

In [ ]:
# Create a ZipFile Object and load train.zip in it
#with ZipFile(os.path.join(data_path, "train.zip"), 'r') as zipObj:
#   # Extract all the contents of zip file in different directory
#   zipObj.extractall()

In [ ]:
study_train = next(os.walk(os.path.join(data_path, "train")))[1]
# load labels in 'train.csv'
# the first column means id
# the second and third columns mean the volume
labels = np.loadtxt(os.path.join(data_path, "train.csv"), delimiter=",",skiprows=1)
labels[0:10]

In [ ]:
class Dataset(object):
    dataset_count = 0

    def __init__(self, directory, subdir):
        # deal with any intervening directories
        while True:
            subdirs = next(os.walk(directory))[1]
            if len(subdirs) == 1:
                directory = os.path.join(directory, subdirs[0])
            else:
                break

        slices = []
        for s in subdirs:
            m = re.match("sax_(\d+)", s)
            if m is not None:
                slices.append(int(m.group(1)))

        slices_map = {}
        first = True
        times = []
        for s in slices:
            files = next(os.walk(os.path.join(directory, "sax_%d" % s)))[2]
            offset = None

            for f in files:
                m = re.match("IM-(\d{4,})-(\d{4})\.dcm", f)
                if m is not None:
                    if first:
                        times.append(int(m.group(2)))
                    if offset is None:
                        offset = int(m.group(1))

            first = False
            slices_map[s] = offset

        self.directory = directory
        self.time = sorted(times)
        self.slices = sorted(slices)
        self.slices_map = slices_map
        Dataset.dataset_count += 1
        self.name = subdir

    def _filename(self, s, t):
        return os.path.join(self.directory,"sax_%d" % s, "IM-%04d-%04d.dcm" % (self.slices_map[s], t))

    def _read_dicom_image(self, filename):
        d = pydicom.read_file(filename)
        img = d.pixel_array
        IMG_PX_SIZE = 64
        resized_img = resize(img, (IMG_PX_SIZE, IMG_PX_SIZE), anti_aliasing=True)
        return np.array(resized_img)

    def _read_all_dicom_images(self):
        f1 = self._filename(self.slices[0], self.time[0])
        d1 = pydicom.read_file(f1)
        (x, y) = d1.PixelSpacing
        (x, y) = (float(x), float(y))
        f2 = self._filename(self.slices[1], self.time[0])
        d2 = pydicom.read_file(f2)

        # try a couple of things to measure distance between slices
        try:
            dist = np.abs(d2.SliceLocation - d1.SliceLocation)
        except AttributeError:
            try:
                dist = d1.SliceThickness
            except AttributeError:
                dist = 8  # better than nothing...

        self.images = np.array([[self._read_dicom_image(self._filename(d, i))
                                 for i in self.time]
                                for d in self.slices])
        self.dist = dist
        self.area_multiplier = x * y

    def load(self):
        self._read_all_dicom_images()

In [ ]:
dset = []
for i,s in enumerate(study_train):
    full_path = os.path.join(data_path, "train", s)
    dset.append(Dataset(full_path, s))
    print("Processing dataset %s..." % dset[i].name)
    p_edv = 0
    p_esv = 0
    try:
        dset[i].load()
        print("Dataset %s processing done." % dset[i].name)
    except Exception as e:
        print("ERROR: Exception %s thrown by dataset %s" % (str(e), dset[i].name))
        print("Omit index: %s" % i)

In [ ]:
# note 337, 437, 463, 499, 234, 393, 334, 305, 279, 416, 41, 123 can not loaded 
# so we just remove them from our training data
omit_subject = [463, 499, 234, 334, 279, 416, 123]
omit_index = [134, 140, 169, 305, 333, 352, 433]

In [ ]:
# refine the whole dataset
study_index = [int(ele) for ele in study_train]
study_index = [study_index[i] for i in range(len(study_index)) if i not in omit_index]
X = []
for ind, val in enumerate(study_train):
    try:
        new_image = dset[ind].images
        X.append(new_image)
    except Exception as e:
        print("ERROR: Exception %s" % str(e))
        print("Stop at index: %s" % val)

In [ ]:
X[3].swapaxes(1,3).shape

In [ ]:
# For example, to simplify this problem, we may just take the average of images
# for each subject
# Of course, you can consider more complicated method to obtain better performance
X_train_average = []
for i in range(len(X)):
    images = X[i].swapaxes(1,3)
    t, s, w, h = X[i].shape
    image_sum = np.zeros([64,64,30])
    for j in range(t):
        image_sum = image_sum + images[j,:,:,:]
            
    image_average = image_sum / t
    X_train_average.append(image_average)
X_train_average

In [ ]:
# Target for example: systole only
label = pd.DataFrame(labels)
label.columns = ['Id','Systole','Diastole']
actual_value = np.round(label.loc[np.array(study_index)-1,'Systole']).astype(int)
Y_train = actual_value

# one-hot encode
label_train = np.zeros([len(Y_train), 600])
for i in range(len(Y_train)):
    value = Y_train.iloc[i]
    label_train[i,value-1] = 1

In [ ]:
# predictor
X_train_average = np.array(X_train_average).reshape([-1, 64, 64, 30])
X_train_average.shape

In [ ]:
#X_train_average.reshape([493,64*64])
#X_train_average[0].reshape([64*64])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_average, label_train, test_size=0.33, random_state=42)
#np.savetxt('X_train.csv', X_train.reshape((330,64*64*30)), delimiter=',')
#np.savetxt('X_test.csv', X_test.reshape((163,64*64*30)), delimiter=',')
#np.savetxt('y_train.csv',y_train, delimiter=',')
#np.savetxt('y_test.csv', y_test, delimiter=',')

In [2]:
#np.save('X_train.npy', X_train)
X_train_load = np.load('X_train.npy')

#np.save('y_train.npy', y_train)
y_train_load = np.load('y_train.npy')

#np.save('X_test.npy', X_test)
X_test_load = np.load('X_test.npy')

#np.save('y_test.npy', y_test)
y_test_load = np.load('y_test.npy')

In [ ]:
#X_train_load = np.loadtxt('X_train.csv', delimiter = ',').reshape(-1,64,64,30)
#X_test = np.loadtxt('X_test.csv', delimiter = ',').reshape(-1,64,64,1)
#y_train = np.loadtxt('y_train.csv', delimiter = ',')
#y_test = np.loadtxt('y_test.csv', delimiter = ',')
#assert X_train_load == X_train

In [3]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
network = input_data(shape=[None, 64, 64, 30], name='input')
network = conv_2d(network, 32, 4, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 4, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 128, activation='relu')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='relu')
network = dropout(network, 0.8)
network = fully_connected(network, 600, activation='softmax')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X_train_load}, {'target': y_train_load}, n_epoch=10,
           validation_set=({'input': X_test_load}, {'target': y_test_load}),
           snapshot_step=100, show_metric=True, run_id='convnet')

Training Step: 59  | total loss: 4.61479 | time: 0.193s
| Adam | epoch: 010 | loss: 4.61479 - acc: 0.0527 -- iter: 320/330
Training Step: 60  | total loss: 4.62947 | time: 1.252s
| Adam | epoch: 010 | loss: 4.62947 - acc: 0.0498 | val_loss: 5.47604 - val_acc: 0.0184 -- iter: 330/330
--


In [ ]:
pred = model.predict({'input': X_test_load})

In [ ]:
pred[:,1]
#np.savetxt('y_result.csv',pred, delimiter=',')